In [1]:
pip install spacy


In [2]:
!python -m spacy download en_core_web_sm


2023-12-28 13:12:56.130070: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 13:12:56.130126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 13:12:56.131575: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-28 13:12:56.139467: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-28 13:12:57.368279: W tensorflow/compiler/tf2

In [3]:
pip install pymupdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 17.3 MB/s eta 0:00:00


In [19]:
import fitz
import spacy
from spacy.matcher import Matcher

def extract_text_from_pdf(pdf_path):
    text = ''
    with fitz.open(pdf_path) as pdf_doc:
        for page_num in range(pdf_doc.page_count):
            page = pdf_doc[page_num]
            text += page.get_text()
    return text

def extract_information(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)

    matcher = Matcher(nlp.vocab)


    email_pattern = [{"LIKE_EMAIL": True}]
    phone_pattern = [{"TEXT": {"REGEX": r'\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b'}}]
    skills_pattern = [{"LOWER": {"IN": ["python", "java", "c++", "javascript", "sql"]}}]
    education_pattern = [{"LOWER": "education"}, {"POS": "ADP"}, {"POS": "NOUN"}]
    experience_pattern = [{"LOWER": {"IN": ["experience", "work"]}}, {"POS": "ADP"}, {"POS": "NOUN"}]

    matcher.add("Email", [email_pattern])
    matcher.add("Phone", [phone_pattern])
    matcher.add("Skills", [skills_pattern])
    matcher.add("Education", [education_pattern])
    matcher.add("Experience", [experience_pattern])

    matches = matcher(doc)

    information = {
        "name": None,
        "email": None,
        "phone": None,
        "skills": [],
        "education": [],
        "experience": [],
    }

    for match_id, start, end in matches:
        match_id_str = nlp.vocab.strings[match_id]
        if match_id_str == "Email":
            information["email"] = doc[start:end].text
        elif match_id_str == "Phone":
            information["phone"] = doc[start:end].text
        elif match_id_str == "Skills":
            information["skills"].append(doc[start:end].text)
        elif match_id_str == "Education":
            information["education"].append(doc[start:end].text)
        elif match_id_str == "Experience":
            information["experience"].append(doc[start:end].text)


    if doc and doc[0].is_alpha:
        information["name"] = doc[0].text

    return information

if __name__ == "__main__":

    resume_path = "/content/AashishaNegi.pdf"


    resume_text = extract_text_from_pdf(resume_path)

    resume_info = extract_information(resume_text)

    print("Name:", resume_info["name"])
    print("Email:", resume_info["email"])
    print("Phone:", resume_info["phone"])
    print("Skills:", resume_info["skills"])
    print("Education:", resume_info["education"])
    print("Experience:", resume_info["experience"])


Name: AASHISHA
Email: aashishanegi2303@gmail.com
Phone: 7003995974
Skills: ['SQL', 'C++', 'Java', 'SQL']
Education: []
Experience: []


In [15]:
import fitz
import spacy

def extract_text_from_pdf(pdf_path):
    text = ''
    with fitz.open(pdf_path) as pdf_doc:
        for page_num in range(pdf_doc.page_count):
            page = pdf_doc[page_num]
            text += page.get_text()
    return text

def extract_information(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)

    information = {
        "name": None,
        "email": None,
        "phone": None,
        "skills": [],
        "education": [],
        "experience": [],
    }

    for ent in doc.ents:
        if ent.label_ == "PERSON" and information["name"] is None:
            information["name"] = ent.text
        elif ent.label_ == "EMAIL" and information["email"] is None:
            information["email"] = ent.text
        elif ent.label_ == "PHONE" and information["phone"] is None:
            information["phone"] = ent.text
        elif ent.label_ == "ORG" and ("education" in ent.text.lower() or "school" in ent.text.lower()):
            information["education"].append(ent.text)
        elif ent.label_ == "ORG" and ("experience" in ent.text.lower() or "work" in ent.text.lower()):
            information["experience"].append(ent.text)

    return information

if __name__ == "__main__":

    resume_path = "/content/AashishaNegi.pdf"

    resume_text = extract_text_from_pdf(resume_path)

    resume_info = extract_information(resume_text)

    print("Name:", resume_info["name"])
    print("Email:", resume_info["email"])
    print("Phone:", resume_info["phone"])
    print("Skills:", resume_info["skills"])
    print("Education:", resume_info["education"])
    print("Experience:", resume_info["experience"])


Name: Cloud Computing
Email: None
Phone: None
Skills: []
Education: ['Council for Indian School Certificate Examination', 'Council for Indian School Certificate Examination']
Experience: []


In [21]:
import fitz
import spacy
from spacy.matcher import Matcher

def extract_text_from_pdf(pdf_path):
    text = ''
    with fitz.open(pdf_path) as pdf_doc:
        for page_num in range(pdf_doc.page_count):
            page = pdf_doc[page_num]
            text += page.get_text()
    return text

def extract_information(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)

    matcher = Matcher(nlp.vocab)

    name_pattern = [{"ENT_TYPE": "PERSON"}]
    email_pattern = [{"LIKE_EMAIL": True}]
    phone_pattern = [{"TEXT": {"REGEX": r'\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b'}}]
    skills_pattern = [{"LOWER": {"IN": ["python", "java", "c++", "javascript", "sql"]}}]

    matcher.add("Name", [name_pattern])
    matcher.add("Email", [email_pattern])
    matcher.add("Phone", [phone_pattern])
    matcher.add("Skills", [skills_pattern])

    matches = matcher(doc)

    information = {
        "name": None,
        "email": None,
        "phone": None,
        "skills": [],
        "education": [],
        "experience": [],
    }

    for ent in doc.ents:
        if ent.label_ == "ORG" and ("education" in ent.text.lower() or "school" in ent.text.lower()):
            information["education"].append(ent.text)
        elif ent.label_ == "ORG" and ("experience" in ent.text.lower() or "work" in ent.text.lower()):
            information["experience"].append(ent.text)

    for match_id, start, end in matches:
        match_id_str = nlp.vocab.strings[match_id]
        if match_id_str == "Email" and information["email"] is None:
            information["email"] = doc[start:end].text
        elif match_id_str == "Phone" and information["phone"] is None:
            information["phone"] = doc[start:end].text
        elif match_id_str == "Skills":
            information["skills"].append(doc[start:end].text)

    if doc and doc[0].is_alpha:
        information["name"] = doc[0].text

    return information

if __name__ == "__main__":

    resume_path = "/content/Utkarsh_Rastogi.pdf"

    resume_text = extract_text_from_pdf(resume_path)

    resume_info = extract_information(resume_text)

    print("Name:", resume_info["name"])
    print("Email:", resume_info["email"])
    print("Phone:", resume_info["phone"])
    print("Skills:", resume_info["skills"])
    print("Education:", resume_info["education"])
    print("Experience:", resume_info["experience"])


Name: UTKARSH
Email: utkarshrastogi010@gmail.com
Phone: 8171538808
Skills: ['SQL', 'C++', 'Python', 'Java', 'SQL', 'C++', 'Python']
Education: ['EDUCATION', 'All India Senior School Certificate Examination', 'Birla Public School', 'All India Secondary School Examination', 'Birla Public School']
Experience: ['EXPERIENCE']


In [20]:
import fitz
import re

def extract_text_from_pdf(pdf_path):
    text = ''
    with fitz.open(pdf_path) as pdf_doc:
        for page_num in range(pdf_doc.page_count):
            page = pdf_doc[page_num]
            text += page.get_text()
    return text

def extract_information(text):
    information = {
        "name": None,
        "email": None,
        "phone": None,
        "education": [],
        "experience": [],
    }

    name_match = re.search(r'^([^\n]+)', text)
    if name_match:
        information["name"] = name_match.group(1).strip()

    email_match = re.search(r'[\w\.-]+@[\w\.-]+', text)
    if email_match:
        information["email"] = email_match.group()

    phone_match = re.search(r'\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b', text)
    if phone_match:
        information["phone"] = phone_match.group()

    education_matches = re.findall(r'Education[:\s]+([^\n]+)', text, re.IGNORECASE)
    information["education"] = [edu.strip() for edu in education_matches]

    experience_matches = re.findall(r'Experience[:\s]+([^\n]+)', text, re.IGNORECASE)
    information["experience"] = [exp.strip() for exp in experience_matches]

    return information

if __name__ == "__main__":

    resume_path = "/content/Utkarsh_Rastogi.pdf"

    resume_text = extract_text_from_pdf(resume_path)

    resume_info = extract_information(resume_text)

    print("Name:", resume_info["name"])
    print("Email:", resume_info["email"])
    print("Phone:", resume_info["phone"])
    print("Education:", resume_info["education"])
    print("Experience:", resume_info["experience"])


Name: UTKARSH RASTOGI
Email: utkarshrastogi010@gmail.com
Phone: 8171538808
Education: ['Bachelor of Technology Computer Science, UPES, Dehradun']
Experience: ['AI Intern', '• Open Source Contributor at Hacktoberfest 2022 & 2023.']
